# Retrieve Dataset IDs and Column Names on NYC Open Data
Author: Mark Bauer

### **Goal: Retrive dataset IDs and column names from NYC Open Data. We will use this CSV to programmatically loop through datasets to search for the word *flood*.**

# Importing Libraries

In [1]:
# importing libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
import requests
import time

Documention for installing watermark: https://github.com/rasbt/watermark.

In [2]:
# performed for reproducibility
%reload_ext watermark
%watermark -t -d -v -p pandas,sodapy

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

pandas: 1.5.1
sodapy: 2.2.0



# Socrata API
I used the Socrata API to retrieve metadata for datasets hosted on NYC Open Data. Documentation can be found here: https://dev.socrata.com/. Additionally, I used sodapy, the python client for the Socrata API, to query the metadata.

We'll use this API to gather all the datasets on NYC Open Data.

### Note:  
`WARNING:root:Requests made without an app_token will be subject to strict throttling limits.`

Read more from the SODA documentation here: https://dev.socrata.com/docs/app-tokens.html

In [3]:
ls

cover-photo.ipynb        metadata-analysis.ipynb
dataset-analysis.ipynb   search-text-flood.ipynb


In [4]:
# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# initialize Socrata object to fetch data
client = Socrata(
    domain=socrata_domain,
    app_token=None,
    timeout=10000
)

print(client)

In [5]:
# Discovery API
url = 'https://api.us.socrata.com/api/catalog/v1?search_context=data.cityofnewyork.us&limit=50000'

# fetch the JSON data from the web
response = requests.get(url)

# parse the JSON response
data_dict = response.json() 

# preview keys    
data_dict.keys() 

dict_keys(['results', 'resultSetSize', 'timings', 'warnings'])

In [6]:
# convert into df
df = pd.DataFrame.from_records(data_dict['results'])

# sanity check
print(df.shape)
df.head()

(3199, 8)


,resource,classification,metadata,permalink,link,owner,creator,preview_image_url
0,"{'name': 'For Hire Vehicles (FHV) - Active', '...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/8wbx-tsch,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
1,"{'name': 'Civil Service List (Active)', 'id': ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/vx8i-nprf,https://data.cityofnewyork.us/City-Government/...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
2,"{'name': 'DOB Job Application Filings', 'id': ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/ic3t-wcy2,https://data.cityofnewyork.us/Housing-Developm...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
3,"{'name': 'TLC New Driver Application Status', ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/dpec-ucu7,https://data.cityofnewyork.us/Transportation/T...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
4,{'name': 'For Hire Vehicles (FHV) - Active Dri...,"{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/xjfq-wh2d,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN


In [7]:
# convert resource key to a dataframe
df = pd.DataFrame.from_records(df['resource'])

# sanity check
print(df.shape)
df.head()

(3199, 27)


,name,id,resource_name,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,...,columns_description,columns_format,download_count,provenance,lens_view_type,lens_display_type,locked,blob_mime_type,hide_from_data_json,publication_date
0,For Hire Vehicles (FHV) - Active,8wbx-tsch,None,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2025-01-24T19:57:07.000Z,...,"[Base Number, Vehicle Year, Date Suspension or...","[{'displayStyle': 'plain', 'align': 'left'}, {...",539625,official,tabular,table,False,None,False,2021-04-05T13:20:47.000Z
1,Civil Service List (Active),vx8i-nprf,None,[],A Civil Service List consists of all candidate...,Department of Citywide Administrative Services...,None,None,dataset,2025-01-24T14:14:49.000Z,...,[A “Sibling Legacy Credit” is additional credi...,"[{'displayStyle': 'plain', 'align': 'left'}, {...",69765,official,tabular,table,False,None,False,2024-01-12T16:15:05.000Z
2,DOB Job Application Filings,ic3t-wcy2,None,[],This dataset contains all job applications sub...,Department of Buildings (DOB),None,None,dataset,2025-01-24T21:10:03.000Z,...,"[Loft Board, 1= Manhattan, 2= Bronx, 3 = Brook...","[{'align': 'right'}, {'align': 'right'}, {'ali...",60242,official,tabular,table,False,None,False,2020-06-22T18:23:35.000Z
3,TLC New Driver Application Status,dpec-ucu7,None,[],THIS DATASET IS UPDATED SEVERAL TIMES PER DAY....,Taxi and Limousine Commission (TLC),None,None,dataset,2025-01-25T11:03:13.000Z,...,[This is the number linked to your application...,"[{'precisionStyle': 'standard', 'noCommas': 't...",39718,official,tabular,table,False,None,False,2019-12-17T18:44:57.000Z
4,For Hire Vehicles (FHV) - Active Drivers,xjfq-wh2d,None,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2025-01-24T19:58:30.000Z,...,"[Driver Name\n\n, Last Date Updated, Last Time...","[{'displayStyle': 'plain', 'align': 'left'}, {...",425132,official,tabular,table,False,None,False,2024-01-11T19:58:17.000Z


In [8]:
# preview columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 3199 non-null   object
 1   id                   3199 non-null   object
 2   resource_name        0 non-null      object
 3   parent_fxf           3199 non-null   object
 4   description          3199 non-null   object
 5   attribution          3129 non-null   object
 6   attribution_link     495 non-null    object
 7   contact_email        0 non-null      object
 8   type                 3199 non-null   object
 9   updatedAt            3199 non-null   object
 10  createdAt            3199 non-null   object
 11  metadata_updated_at  3199 non-null   object
 12  data_updated_at      3028 non-null   object
 13  page_views           3199 non-null   object
 14  columns_name         3199 non-null   object
 15  columns_field_name   3199 non-null   object
 16  column

In [9]:
# review dataset types, we only want datasets
df['type'].value_counts()

dataset          2570
map               287
file              167
href              147
filter             24
story               2
chart               1
visualization       1
Name: type, dtype: int64

In [10]:
# we only want datasets
df = (
    df
    .loc[df['type'] == 'dataset']
    .reset_index(drop=True)
)

# sanity check
print(df.shape)
df.head()

(2570, 27)


,name,id,resource_name,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,...,columns_description,columns_format,download_count,provenance,lens_view_type,lens_display_type,locked,blob_mime_type,hide_from_data_json,publication_date
0,For Hire Vehicles (FHV) - Active,8wbx-tsch,None,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2025-01-24T19:57:07.000Z,...,"[Base Number, Vehicle Year, Date Suspension or...","[{'displayStyle': 'plain', 'align': 'left'}, {...",539625,official,tabular,table,False,None,False,2021-04-05T13:20:47.000Z
1,Civil Service List (Active),vx8i-nprf,None,[],A Civil Service List consists of all candidate...,Department of Citywide Administrative Services...,None,None,dataset,2025-01-24T14:14:49.000Z,...,[A “Sibling Legacy Credit” is additional credi...,"[{'displayStyle': 'plain', 'align': 'left'}, {...",69765,official,tabular,table,False,None,False,2024-01-12T16:15:05.000Z
2,DOB Job Application Filings,ic3t-wcy2,None,[],This dataset contains all job applications sub...,Department of Buildings (DOB),None,None,dataset,2025-01-24T21:10:03.000Z,...,"[Loft Board, 1= Manhattan, 2= Bronx, 3 = Brook...","[{'align': 'right'}, {'align': 'right'}, {'ali...",60242,official,tabular,table,False,None,False,2020-06-22T18:23:35.000Z
3,TLC New Driver Application Status,dpec-ucu7,None,[],THIS DATASET IS UPDATED SEVERAL TIMES PER DAY....,Taxi and Limousine Commission (TLC),None,None,dataset,2025-01-25T11:03:13.000Z,...,[This is the number linked to your application...,"[{'precisionStyle': 'standard', 'noCommas': 't...",39718,official,tabular,table,False,None,False,2019-12-17T18:44:57.000Z
4,For Hire Vehicles (FHV) - Active Drivers,xjfq-wh2d,None,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2025-01-24T19:58:30.000Z,...,"[Driver Name\n\n, Last Date Updated, Last Time...","[{'displayStyle': 'plain', 'align': 'left'}, {...",425132,official,tabular,table,False,None,False,2024-01-11T19:58:17.000Z


In [11]:
# sanity check view type
df['lens_view_type'].value_counts()

tabular    2570
Name: lens_view_type, dtype: int64

In [12]:
# sort df by download count ascending order
df = (
    df
    .sort_values(by='id')
    .reset_index(drop=True)
)

# sanity check
print(df.shape)
df.head()

(2570, 27)


,name,id,resource_name,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,...,columns_description,columns_format,download_count,provenance,lens_view_type,lens_display_type,locked,blob_mime_type,hide_from_data_json,publication_date
0,2016-2017 Student Discipline Annual Report - ELL,22rr-ujq3,None,[],Student discipline annual report for English l...,Department of Education (DOE),None,None,dataset,2024-11-26T23:15:04.000Z,...,"[, , , , , , , , , , , , ]","[{}, {}, {}, {'decimalSeparator': '.', 'groupS...",752,official,tabular,table,False,None,False,2018-06-07T22:21:57.000Z
1,2020 DOE High School Directory,23z9-6uk9,None,[],Directory of Department of Education High Scho...,Department of Education (DOE),None,None,dataset,2024-11-26T23:14:03.000Z,...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {...",3176,official,tabular,table,False,None,False,2019-10-23T22:16:05.000Z
2,DSNY Special Waste Drop-off Sites,242c-ru4i,None,[],Location of DSNY Special Waste Drop-Off Sites....,Department of Sanitation,None,None,dataset,2025-01-25T11:02:19.000Z,...,"[, In ArcGIS, the FID is a system-managed valu...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {...",2115,official,tabular,table,False,None,False,2020-07-24T03:09:55.000Z
3,Five Year Plan Summary by Capital Category,24nr-gahi,None,[],Five year plan summary cost by capital category.,NYC School Construction Authority (SCA),None,None,dataset,2025-01-10T16:06:06.000Z,...,"[, , , , , , , , ]","[{}, {}, {}, {}, {}, {}, {}, {}, {}]",1704,official,tabular,table,False,None,False,2023-12-27T20:28:30.000Z
4,2018-2019 Local Law 120 Public School Athletic...,24ts-hbqj,None,[],Public Schools Athletic League (PSAL) particip...,Department of Education (DOE),None,None,dataset,2024-11-26T23:08:02.000Z,...,"[, , , , , , , , , , , , , , , , , , , , , , ,...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {...",313,official,tabular,table,False,None,False,2022-03-02T21:19:56.000Z


In [13]:
field_names_df = df.loc[:, ['id', 'columns_field_name']].explode('columns_field_name')
field_types_df = df.loc[:, ['id', 'columns_datatype']].explode('columns_datatype')

df = (
    pd
    .concat([field_names_df, field_types_df.drop(columns=['id'])], axis=1)
    .reset_index(drop=True)
)

# sanity check
print(df.shape)
df.head()

(59030, 3)


,id,columns_field_name,columns_datatype
0,22rr-ujq3,non_ell_expulsions,Text
1,22rr-ujq3,location_name,Text
2,22rr-ujq3,location_category,Text
3,22rr-ujq3,administrative_district,Number
4,22rr-ujq3,ell_removals,Text


In [14]:
df = (
    df
    .loc[df['columns_datatype'] == 'Text']
    .reset_index(drop=True)
)

# sanity check
print(df.shape)
df.head()

(35580, 3)


,id,columns_field_name,columns_datatype
0,22rr-ujq3,non_ell_expulsions,Text
1,22rr-ujq3,location_name,Text
2,22rr-ujq3,location_category,Text
3,22rr-ujq3,ell_removals,Text
4,22rr-ujq3,ell_expulsions,Text


In [15]:
# sanity check
df['columns_datatype'].value_counts()

Text    35580
Name: columns_datatype, dtype: int64

In [16]:
# number of dataset
df['id'].nunique()

2434

In [17]:
# save as a CSV file
df.to_csv('../data/dataset-ids-columns.csv', index=False)

In [18]:
# sanity check
%ls ../data/

dataset-ids-columns.csv  flood-datasets.csv
